# From miniJPAS query to catalog

#### This ntbk will compute fluxes and errors taking in account the zero points errors from the calibration for each Tile, and will set as NaN all the negative fluxes. 
##### Why? For very faint objects, with fluxes at the noise level in bands different than the detection one, it is possible to obtain negative fluxes. This is not a problem and is consistent with the fact of measuring within the noise. (Bonoli+20)

## Imports

In [11]:
import numpy as np
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.io import fits
from astropy.table import QTable

In [12]:
#Query FLUXES and MAG
#SELECT mag.TILE_ID, mag.NUMBER, mag.ALPHA_J2000 AS RA, mag.DELTA_J2000 AS DEC, mag.CLASS_STAR, mag.MAG_AUTO[minijpas::rSDSS] as rMAGsdss, calib.ZPT as ZeroP, calib.ERRZPT as ZeroP_err, mag.MAG_AUTO[minijpas::J0378] AS magAB_J0378, mag.MAG_ERR_AUTO[minijpas::J0378] AS magAB_J0378_err, mag.FLAGS[minijpas::J0378] as SEFLAG_J0378, mag.MASK_FLAGS[minijpas::J0378] as MAFLAG_J0378, mag.MAG_AUTO[minijpas::J0390] AS magAB_J0390, mag.MAG_ERR_AUTO[minijpas::J0390] AS magAB_J0390_err, mag.FLAGS[minijpas::J0390] as SEFLAG_J0390, mag.MASK_FLAGS[minijpas::J0390] as MAFLAG_J0390, mag.MAG_AUTO[minijpas::J0400] AS magAB_J0400, mag.MAG_ERR_AUTO[minijpas::J0400] AS magAB_J0400_err, mag.FLAGS[minijpas::J0400] as SEFLAG_J0400, mag.MASK_FLAGS[minijpas::J0400] as MAFLAG_J0400, mag.MAG_AUTO[minijpas::J0410] AS magAB_J0410, mag.MAG_ERR_AUTO[minijpas::J0410] AS magAB_J0410_err, mag.FLAGS[minijpas::J0410] as SEFLAG_J0410, mag.MASK_FLAGS[minijpas::J0410] as MAFLAG_J0410, mag.MAG_AUTO[minijpas::J0420] AS magAB_J0420, mag.MAG_ERR_AUTO[minijpas::J0420] AS magAB_J0420_err, mag.FLAGS[minijpas::J0420] as SEFLAG_J0420, mag.MASK_FLAGS[minijpas::J0420] as MAFLAG_J0420, mag.MAG_AUTO[minijpas::J0430] AS magAB_J0430, mag.MAG_ERR_AUTO[minijpas::J0430] AS magAB_J0430_err, mag.FLAGS[minijpas::J0430] as SEFLAG_J0430, mag.MASK_FLAGS[minijpas::J0430] as MAFLAG_J0430, mag.MAG_AUTO[minijpas::J0440] AS magAB_J0440, mag.MAG_ERR_AUTO[minijpas::J0440] AS magAB_J0440_err, mag.FLAGS[minijpas::J0440] as SEFLAG_J0440, mag.MASK_FLAGS[minijpas::J0440] as MAFLAG_J0440, mag.MAG_AUTO[minijpas::J0450] AS magAB_J0450, mag.MAG_ERR_AUTO[minijpas::J0450] AS magAB_J0450_err, mag.FLAGS[minijpas::J0450] as SEFLAG_J0450, mag.MASK_FLAGS[minijpas::J0450] as MAFLAG_J0450, mag.MAG_AUTO[minijpas::J0460] AS magAB_J0460, mag.MAG_ERR_AUTO[minijpas::J0460] AS magAB_J0460_err, mag.FLAGS[minijpas::J0460] as SEFLAG_J0460, mag.MASK_FLAGS[minijpas::J0460] as MAFLAG_J0460, mag.MAG_AUTO[minijpas::J0470] AS magAB_J0470, mag.MAG_ERR_AUTO[minijpas::J0470] AS magAB_J0470_err, mag.FLAGS[minijpas::J0470] as SEFLAG_J0470, mag.MASK_FLAGS[minijpas::J0470] as MAFLAG_J0470, mag.MAG_AUTO[minijpas::J0480] AS magAB_J0480, mag.MAG_ERR_AUTO[minijpas::J0480] AS magAB_J0480_err, mag.FLAGS[minijpas::J0480] as SEFLAG_J0480, mag.MASK_FLAGS[minijpas::J0480] as MAFLAG_J0480, mag.MAG_AUTO[minijpas::J0490] AS magAB_J0490, mag.MAG_ERR_AUTO[minijpas::J0490] AS magAB_J0490_err, mag.FLAGS[minijpas::J0490] as SEFLAG_J0490, mag.MASK_FLAGS[minijpas::J0490] as MAFLAG_J0490, mag.MAG_AUTO[minijpas::J0500] AS magAB_J0500, mag.MAG_ERR_AUTO[minijpas::J0500] AS magAB_J0500_err, mag.FLAGS[minijpas::J0500] as SEFLAG_J0500, mag.MASK_FLAGS[minijpas::J0500] as MAFLAG_J0500, mag.MAG_AUTO[minijpas::J0510] AS magAB_J0510, mag.MAG_ERR_AUTO[minijpas::J0510] AS magAB_J0510_err, mag.FLAGS[minijpas::J0510] as SEFLAG_J0510, mag.MASK_FLAGS[minijpas::J0510] as MAFLAG_J0510, mag.MAG_AUTO[minijpas::J0520] AS magAB_J0520, mag.MAG_ERR_AUTO[minijpas::J0520] AS magAB_J0520_err, mag.FLAGS[minijpas::J0520] as SEFLAG_J0520, mag.MASK_FLAGS[minijpas::J0520] as MAFLAG_J0520, mag.MAG_AUTO[minijpas::J0530] AS magAB_J0530, mag.MAG_ERR_AUTO[minijpas::J0530] AS magAB_J0530_err, mag.FLAGS[minijpas::J0530] as SEFLAG_J0530, mag.MASK_FLAGS[minijpas::J0530] as MAFLAG_J0530, mag.MAG_AUTO[minijpas::J0540] AS magAB_J0540, mag.MAG_ERR_AUTO[minijpas::J0540] AS magAB_J0540_err, mag.FLAGS[minijpas::J0540] as SEFLAG_J0540, mag.MASK_FLAGS[minijpas::J0540] as MAFLAG_J0540, mag.MAG_AUTO[minijpas::J0550] AS magAB_J0550, mag.MAG_ERR_AUTO[minijpas::J0550] AS magAB_J0550_err, mag.FLAGS[minijpas::J0550] as SEFLAG_J0550, mag.MASK_FLAGS[minijpas::J0550] as MAFLAG_J0550, mag.MAG_AUTO[minijpas::J0560] AS magAB_J0560, mag.MAG_ERR_AUTO[minijpas::J0560] AS magAB_J0560_err, mag.FLAGS[minijpas::J0560] as SEFLAG_J0560, mag.MASK_FLAGS[minijpas::J0560] as MAFLAG_J0560, mag.MAG_AUTO[minijpas::J0570] AS magAB_J0570, mag.MAG_ERR_AUTO[minijpas::J0570] AS magAB_J0570_err, mag.FLAGS[minijpas::J0570] as SEFLAG_J0570, mag.MASK_FLAGS[minijpas::J0570] as MAFLAG_J0570, mag.MAG_AUTO[minijpas::J0580] AS magAB_J0580, mag.MAG_ERR_AUTO[minijpas::J0580] AS magAB_J0580_err, mag.FLAGS[minijpas::J0580] as SEFLAG_J0580, mag.MASK_FLAGS[minijpas::J0570] as MAFLAG_J0570, mag.MAG_AUTO[minijpas::J0590] AS magAB_J0590, mag.MAG_ERR_AUTO[minijpas::J0590] AS magAB_J0590_err, mag.FLAGS[minijpas::J0590] as SEFLAG_J0590, mag.MASK_FLAGS[minijpas::J0590] as MAFLAG_J0590, mag.MAG_AUTO[minijpas::J0600] AS magAB_J0600, mag.MAG_ERR_AUTO[minijpas::J0600] AS magAB_J0600_err, mag.FLAGS[minijpas::J0600] as SEFLAG_J0600, mag.MASK_FLAGS[minijpas::J0600] as MAFLAG_J0600, mag.MAG_AUTO[minijpas::J0610] AS magAB_J0610, mag.MAG_ERR_AUTO[minijpas::J0610] AS magAB_J0610_err, mag.FLAGS[minijpas::J0610] as SEFLAG_J0610, mag.MASK_FLAGS[minijpas::J0610] as MAFLAG_J0610, mag.MAG_AUTO[minijpas::J0620] AS magAB_J0620, mag.MAG_ERR_AUTO[minijpas::J0620] AS magAB_J0620_err, mag.FLAGS[minijpas::J0620] as SEFLAG_J0620, mag.MASK_FLAGS[minijpas::J0620] as MAFLAG_J0620, mag.MAG_AUTO[minijpas::J0630] AS magAB_J0630, mag.MAG_ERR_AUTO[minijpas::J0630] AS magAB_J0630_err, mag.FLAGS[minijpas::J0630] as SEFLAG_J0630, mag.MASK_FLAGS[minijpas::J0630] as MAFLAG_J0630, mag.MAG_AUTO[minijpas::J0640] AS magAB_J0640, mag.MAG_ERR_AUTO[minijpas::J0640] AS magAB_J0640_err, mag.FLAGS[minijpas::J0640] as SEFLAG_J0640, mag.MASK_FLAGS[minijpas::J0640] as MAFLAG_J0640, mag.MAG_AUTO[minijpas::J0650] AS magAB_J0650, mag.MAG_ERR_AUTO[minijpas::J0650] AS magAB_J0650_err, mag.FLAGS[minijpas::J0650] as SEFLAG_J0650, mag.MASK_FLAGS[minijpas::J0650] as MAFLAG_J0650, mag.MAG_AUTO[minijpas::J0660] AS magAB_J0660, mag.MAG_ERR_AUTO[minijpas::J0660] AS magAB_J0660_err, mag.FLAGS[minijpas::J0660] as SEFLAG_J0660, mag.MASK_FLAGS[minijpas::J0660] as MAFLAG_J0660, mag.MAG_AUTO[minijpas::J0670] AS magAB_J0670, mag.MAG_ERR_AUTO[minijpas::J0670] AS magAB_J0670_err, mag.FLAGS[minijpas::J0670] as SEFLAG_J0670, mag.MASK_FLAGS[minijpas::J0670] as MAFLAG_J0670, mag.MAG_AUTO[minijpas::J0680] AS magAB_J0680, mag.MAG_ERR_AUTO[minijpas::J0680] AS magAB_J0680_err, mag.FLAGS[minijpas::J0680] as SEFLAG_J0680, mag.MASK_FLAGS[minijpas::J0680] as MAFLAG_J0680, mag.MAG_AUTO[minijpas::J0690] AS magAB_J0690, mag.MAG_ERR_AUTO[minijpas::J0690] AS magAB_J0690_err, mag.FLAGS[minijpas::J0690] as SEFLAG_J0690, mag.MASK_FLAGS[minijpas::J0690] as MAFLAG_J0690, mag.MAG_AUTO[minijpas::J0700] AS magAB_J0700, mag.MAG_ERR_AUTO[minijpas::J0700] AS magAB_J0700_err, mag.FLAGS[minijpas::J0700] as SEFLAG_J0700, mag.MASK_FLAGS[minijpas::J0700] as MAFLAG_J0700, mag.MAG_AUTO[minijpas::J0710] AS magAB_J0710, mag.MAG_ERR_AUTO[minijpas::J0710] AS magAB_J0710_err, mag.FLAGS[minijpas::J0710] as SEFLAG_J0710, mag.MASK_FLAGS[minijpas::J0710] as MAFLAG_J0710, mag.MAG_AUTO[minijpas::J0720] AS magAB_J0720, mag.MAG_ERR_AUTO[minijpas::J0720] AS magAB_J0720_err, mag.FLAGS[minijpas::J0720] as SEFLAG_J0720, mag.MASK_FLAGS[minijpas::J0720] as MAFLAG_J0720, mag.MAG_AUTO[minijpas::J0730] AS magAB_J0730, mag.MAG_ERR_AUTO[minijpas::J0730] AS magAB_J0730_err, mag.FLAGS[minijpas::J0730] as SEFLAG_J0730, mag.MASK_FLAGS[minijpas::J0730] as MAFLAG_J0730, mag.MAG_AUTO[minijpas::J0740] AS magAB_J0740, mag.MAG_ERR_AUTO[minijpas::J0740] AS magAB_J0740_err, mag.FLAGS[minijpas::J0740] as SEFLAG_J0740, mag.MASK_FLAGS[minijpas::J0740] as MAFLAG_J0740, mag.MAG_AUTO[minijpas::J0750] AS magAB_J0750, mag.MAG_ERR_AUTO[minijpas::J0750] AS magAB_J0750_err, mag.FLAGS[minijpas::J0750] as SEFLAG_J0750, mag.MASK_FLAGS[minijpas::J0750] as MAFLAG_J0750, mag.MAG_AUTO[minijpas::J0760] AS magAB_J0760, mag.MAG_ERR_AUTO[minijpas::J0760] AS magAB_J0760_err, mag.FLAGS[minijpas::J0760] as SEFLAG_J0760, mag.MASK_FLAGS[minijpas::J0760] as MAFLAG_J0760, mag.MAG_AUTO[minijpas::J0770] AS magAB_J0770, mag.MAG_ERR_AUTO[minijpas::J0770] AS magAB_J0770_err, mag.FLAGS[minijpas::J0770] as SEFLAG_J0770, mag.MASK_FLAGS[minijpas::J0770] as MAFLAG_J0770, mag.MAG_AUTO[minijpas::J0780] AS magAB_J0780, mag.MAG_ERR_AUTO[minijpas::J0780] AS magAB_J0780_err, mag.FLAGS[minijpas::J0780] as SEFLAG_J0780, mag.MASK_FLAGS[minijpas::J0780] as MAFLAG_J0780, mag.MAG_AUTO[minijpas::J0790] AS magAB_J0790, mag.MAG_ERR_AUTO[minijpas::J0790] AS magAB_J0790_err, mag.FLAGS[minijpas::J0790] as SEFLAG_J0790, mag.MASK_FLAGS[minijpas::J0790] as MAFLAG_J0790, mag.MAG_AUTO[minijpas::J0800] AS magAB_J0800, mag.MAG_ERR_AUTO[minijpas::J0800] AS magAB_J0800_err, mag.FLAGS[minijpas::J0800] as SEFLAG_J0800, mag.MASK_FLAGS[minijpas::J0800] as MAFLAG_J0800, mag.MAG_AUTO[minijpas::J0810] AS magAB_J0810, mag.MAG_ERR_AUTO[minijpas::J0810] AS magAB_J0810_err, mag.FLAGS[minijpas::J0810] as SEFLAG_J0810, mag.MASK_FLAGS[minijpas::J0810] as MAFLAG_J0810, mag.MAG_AUTO[minijpas::J0820] AS magAB_J0820, mag.MAG_ERR_AUTO[minijpas::J0820] AS magAB_J0820_err, mag.FLAGS[minijpas::J0820] as SEFLAG_J0820, mag.MASK_FLAGS[minijpas::J0820] as MAFLAG_J0820, mag.MAG_AUTO[minijpas::J0830] AS magAB_J0830, mag.MAG_ERR_AUTO[minijpas::J0830] AS magAB_J0830_err, mag.FLAGS[minijpas::J0830] as SEFLAG_J0830, mag.MASK_FLAGS[minijpas::J0830] as MAFLAG_J0830, mag.MAG_AUTO[minijpas::J0840] AS magAB_J0840, mag.MAG_ERR_AUTO[minijpas::J0840] AS magAB_J0840_err, mag.FLAGS[minijpas::J0840] as SEFLAG_J0840, mag.MASK_FLAGS[minijpas::J0840] as MAFLAG_J0840, mag.MAG_AUTO[minijpas::J0850] AS magAB_J0850, mag.MAG_ERR_AUTO[minijpas::J0850] AS magAB_J0850_err, mag.FLAGS[minijpas::J0850] as SEFLAG_J0850, mag.MASK_FLAGS[minijpas::J0850] as MAFLAG_J0850, mag.MAG_AUTO[minijpas::J0860] AS magAB_J0860, mag.MAG_ERR_AUTO[minijpas::J0860] AS magAB_J0860_err, mag.FLAGS[minijpas::J0860] as SEFLAG_J0860, mag.MASK_FLAGS[minijpas::J0860] as MAFLAG_J0860, mag.MAG_AUTO[minijpas::J0870] AS magAB_J0870, mag.MAG_ERR_AUTO[minijpas::J0870] AS magAB_J0870_err, mag.FLAGS[minijpas::J0870] as SEFLAG_J0870, mag.MASK_FLAGS[minijpas::J0870] as MAFLAG_J0870, mag.MAG_AUTO[minijpas::J0880] AS magAB_J0880, mag.MAG_ERR_AUTO[minijpas::J0880] AS magAB_J0880_err, mag.FLAGS[minijpas::J0880] as SEFLAG_J0880, mag.MASK_FLAGS[minijpas::J0880] as MAFLAG_J0880, mag.MAG_AUTO[minijpas::J0890] AS magAB_J0890, mag.MAG_ERR_AUTO[minijpas::J0890] AS magAB_J0890_err, mag.FLAGS[minijpas::J0890] as SEFLAG_J0890, mag.MASK_FLAGS[minijpas::J0890] as MAFLAG_J0890, mag.MAG_AUTO[minijpas::J0900] AS magAB_J0900, mag.MAG_ERR_AUTO[minijpas::J0900] AS magAB_J0900_err, mag.FLAGS[minijpas::J0900] as SEFLAG_J0900, mag.MASK_FLAGS[minijpas::J0900] as MAFLAG_J0900, mag.MAG_AUTO[minijpas::J0910] AS magAB_J0910, mag.MAG_ERR_AUTO[minijpas::J0910] AS magAB_J0910_err, mag.FLAGS[minijpas::J0910] as SEFLAG_J0910, mag.MASK_FLAGS[minijpas::J0910] as MAFLAG_J0910, mag.MAG_AUTO[minijpas::J1007] AS magAB_J1007, mag.MAG_ERR_AUTO[minijpas::J1007] AS magAB_J1007_err, mag.FLAGS[minijpas::J1007] as SEFLAG_J1007, mag.MASK_FLAGS[minijpas::J1007] as MAFLAG_J1007, mag.MAG_AUTO[minijpas::gSDSS] AS magAB_gSDSS, mag.MAG_ERR_AUTO[minijpas::gSDSS] AS magAB_gSDSS_err, mag.FLAGS[minijpas::gSDSS] as SEFLAG_gSDSS, mag.MASK_FLAGS[minijpas::gSDSS] as MAFLAG_gSDSS, mag.MAG_AUTO[minijpas::iSDSS] AS magAB_iSDSS, mag.MAG_ERR_AUTO[minijpas::iSDSS] AS magAB_iSDSS_err, mag.FLAGS[minijpas::iSDSS] as SEFLAG_iSDSS, mag.MASK_FLAGS[minijpas::iSDSS] as MAFLAG_iSDSS, mag.MAG_AUTO[minijpas::rSDSS] AS magAB_rSDSS, mag.MAG_ERR_AUTO[minijpas::rSDSS] AS magAB_rSDSS_err, mag.FLAGS[minijpas::rSDSS] as SEFLAG_rSDSS, mag.MASK_FLAGS[minijpas::rSDSS] as MAFLAG_rSDSS, mag.MAG_AUTO[minijpas::uJAVA] AS magAB_uJAVA, mag.MAG_ERR_AUTO[minijpas::uJAVA] AS magAB_uJAVA_err, mag.FLAGS[minijpas::uJAVA] as SEFLAG_uJAVA, mag.MASK_FLAGS[minijpas::uJAVA] as MAFLAG_uJAVA FROM minijpas.MagABDualObj AS mag LEFT JOIN minijpas.TileImage as calib ON (mag.TILE_ID = calib.TILE_ID) WHERE mag.MAG_AUTO[minijpas::rSDSS] < 23.6 

In [13]:
#Reading the fits from the Query

In [14]:
with fits.open('./'+'MAG_AUTO_23.fits', memmap=True) as hdul:
    #hdul.info()
    miniJPAS_or = QTable(hdul[1].data)

In [15]:
filters_names=miniJPAS_or.colnames[8::4]
errors_names=[fil + '_err' for fil in filters_names]

In [16]:
#Change negative values for Nan

In [17]:
def Negative2Nan(table,colname):
    temp_table=table.copy()
    temp_table[colname][np.where(table[colname]<0)]=np.nan
    temp_table[colname][np.where(table[colname]==99)]=np.nan
    return temp_table

In [18]:
miniJPAS=miniJPAS_or.copy()
for fil in filters_names:
    miniJPAS=Negative2Nan(miniJPAS,fil)
for fil in errors_names:
    miniJPAS=Negative2Nan(miniJPAS,fil)

In [19]:
#Magnitudes AB to mJy

In [20]:
def ab_to_mjy(m,er):
    fv= 10**((m-8.9)/-2.5) * u.Jy
    er= abs((np.log(10)*10**((m-8.9)/-2.5))/-2.5)*er * u.Jy
    return fv.to(u.mJy),er.to(u.mJy)

In [21]:
for fil in filters_names:
    flux,err=ab_to_mjy(miniJPAS[fil],miniJPAS[fil+'_err'])
    miniJPAS.add_column(flux, name='flux_'+fil[6:])
    miniJPAS.add_column(err, name='flux_'+fil[6:]+'_err')

In [22]:
#Adding zero point errors in quadrature

In [23]:
er_zero_flux=ab_to_mjy(miniJPAS['ZeroP'],miniJPAS['ZeroP_err'])[1]

In [24]:
flux_names=miniJPAS.colnames[244::2]

In [25]:
mean=np.array([])
for fil in flux_names:
    mean=np.append(mean,np.nanmean(miniJPAS[fil+'_err']/miniJPAS[fil]))
np.nanmean(mean)

<Quantity 2.31598052>

In [26]:
miniJPAS_flux=miniJPAS.copy()
for fil in flux_names:
    miniJPAS_flux[fil+'_err']=np.sqrt(miniJPAS[fil+'_err']**2+er_zero_flux**2)

In [27]:
mean=np.array([])
for fil in flux_names:
    mean=np.append(mean,np.nanmean(miniJPAS_flux[fil+'_err']/miniJPAS_flux[fil]))
np.nanmean(mean)

<Quantity 2.31632444>

In [28]:
#ToSave

In [31]:
tosave=['TILE_ID','NUMBER','RA','DEC','CLASS_STAR','rMAGsdss','flux_J0378','flux_J0378_err','SEFLAG_J0378','MAFLAG_J0378','flux_J0390','flux_J0390_err','SEFLAG_J0390','MAFLAG_J0390','flux_J0400','flux_J0400_err','SEFLAG_J0400','MAFLAG_J0400','flux_J0410','flux_J0410_err','SEFLAG_J0410','MAFLAG_J0410','flux_J0420','flux_J0420_err','SEFLAG_J0420','MAFLAG_J0420','flux_J0430','flux_J0430_err','SEFLAG_J0430','MAFLAG_J0430','flux_J0440','flux_J0440_err','SEFLAG_J0440','MAFLAG_J0440','flux_J0450','flux_J0450_err','SEFLAG_J0450','MAFLAG_J0450','flux_J0460','flux_J0460_err','SEFLAG_J0460','MAFLAG_J0460','flux_J0470','flux_J0470_err','SEFLAG_J0470','MAFLAG_J0470','flux_J0480','flux_J0480_err','SEFLAG_J0480','MAFLAG_J0480','flux_J0490','flux_J0490_err','SEFLAG_J0490','MAFLAG_J0490','flux_J0500','flux_J0500_err','SEFLAG_J0500','MAFLAG_J0500','flux_J0510','flux_J0510_err','SEFLAG_J0510','MAFLAG_J0510','flux_J0520','flux_J0520_err','SEFLAG_J0520','MAFLAG_J0520','flux_J0530','flux_J0530_err','SEFLAG_J0530','MAFLAG_J0530','flux_J0540','flux_J0540_err','SEFLAG_J0540','MAFLAG_J0540','flux_J0550','flux_J0550_err','SEFLAG_J0550','MAFLAG_J0550','flux_J0560','flux_J0560_err','SEFLAG_J0560','MAFLAG_J0560','flux_J0570','flux_J0570_err','SEFLAG_J0570','MAFLAG_J0570','flux_J0580','flux_J0580_err','SEFLAG_J0580','MAFLAG_J0570_1','flux_J0590','flux_J0590_err','SEFLAG_J0590','MAFLAG_J0590','flux_J0600','flux_J0600_err','SEFLAG_J0600','MAFLAG_J0600','flux_J0610','flux_J0610_err','SEFLAG_J0610','MAFLAG_J0610','flux_J0620','flux_J0620_err','SEFLAG_J0620','MAFLAG_J0620','flux_J0630','flux_J0630_err','SEFLAG_J0630','MAFLAG_J0630','flux_J0640','flux_J0640_err','SEFLAG_J0640','MAFLAG_J0640','flux_J0650','flux_J0650_err','SEFLAG_J0650','MAFLAG_J0650','flux_J0660','flux_J0660_err','SEFLAG_J0660','MAFLAG_J0660','flux_J0670','flux_J0670_err','SEFLAG_J0670','MAFLAG_J0670','flux_J0680','flux_J0680_err','SEFLAG_J0680','MAFLAG_J0680','flux_J0690','flux_J0690_err','SEFLAG_J0690','MAFLAG_J0690','flux_J0700','flux_J0700_err','SEFLAG_J0700','MAFLAG_J0700','flux_J0710','flux_J0710_err','SEFLAG_J0710','MAFLAG_J0710','flux_J0720','flux_J0720_err','SEFLAG_J0720','MAFLAG_J0720','flux_J0730','flux_J0730_err','SEFLAG_J0730','MAFLAG_J0730','flux_J0740','flux_J0740_err','SEFLAG_J0740','MAFLAG_J0740','flux_J0750','flux_J0750_err','SEFLAG_J0750','MAFLAG_J0750','flux_J0760','flux_J0760_err','SEFLAG_J0760','MAFLAG_J0760','flux_J0770','flux_J0770_err','SEFLAG_J0770','MAFLAG_J0770','flux_J0780','flux_J0780_err','SEFLAG_J0780','MAFLAG_J0780','flux_J0790','flux_J0790_err','SEFLAG_J0790','MAFLAG_J0790','flux_J0800','flux_J0800_err','SEFLAG_J0800','MAFLAG_J0800','flux_J0810','flux_J0810_err','SEFLAG_J0810','MAFLAG_J0810','flux_J0820','flux_J0820_err','SEFLAG_J0820','MAFLAG_J0820','flux_J0830','flux_J0830_err','SEFLAG_J0830','MAFLAG_J0830','flux_J0840','flux_J0840_err','SEFLAG_J0840','MAFLAG_J0840','flux_J0850','flux_J0850_err','SEFLAG_J0850','MAFLAG_J0850','flux_J0860','flux_J0860_err','SEFLAG_J0860','MAFLAG_J0860','flux_J0870','flux_J0870_err','SEFLAG_J0870','MAFLAG_J0870','flux_J0880','flux_J0880_err','SEFLAG_J0880','MAFLAG_J0880','flux_J0890','flux_J0890_err','SEFLAG_J0890','MAFLAG_J0890','flux_J0900','flux_J0900_err','SEFLAG_J0900','MAFLAG_J0900','flux_J0910','flux_J0910_err','SEFLAG_J0910','MAFLAG_J0910','flux_J1007','flux_J1007_err','SEFLAG_J1007','MAFLAG_J1007','flux_gSDSS','flux_gSDSS_err','SEFLAG_gSDSS','MAFLAG_gSDSS','flux_iSDSS','flux_iSDSS_err','SEFLAG_iSDSS','MAFLAG_iSDSS','flux_rSDSS','flux_rSDSS_err','SEFLAG_rSDSS','MAFLAG_rSDSS','flux_uJAVA','flux_uJAVA_err','SEFLAG_uJAVA','MAFLAG_uJAVA']

In [32]:
miniJPAS_flux[tosave]

TILE_ID,NUMBER,RA,DEC,CLASS_STAR,rMAGsdss,flux_J0378,flux_J0378_err,SEFLAG_J0378,MAFLAG_J0378,flux_J0390,flux_J0390_err,SEFLAG_J0390,MAFLAG_J0390,flux_J0400,flux_J0400_err,SEFLAG_J0400,MAFLAG_J0400,flux_J0410,flux_J0410_err,SEFLAG_J0410,MAFLAG_J0410,flux_J0420,flux_J0420_err,SEFLAG_J0420,MAFLAG_J0420,flux_J0430,flux_J0430_err,SEFLAG_J0430,MAFLAG_J0430,flux_J0440,flux_J0440_err,SEFLAG_J0440,MAFLAG_J0440,flux_J0450,flux_J0450_err,SEFLAG_J0450,MAFLAG_J0450,flux_J0460,flux_J0460_err,SEFLAG_J0460,MAFLAG_J0460,flux_J0470,flux_J0470_err,SEFLAG_J0470,MAFLAG_J0470,flux_J0480,flux_J0480_err,SEFLAG_J0480,MAFLAG_J0480,flux_J0490,flux_J0490_err,SEFLAG_J0490,MAFLAG_J0490,flux_J0500,flux_J0500_err,SEFLAG_J0500,MAFLAG_J0500,flux_J0510,flux_J0510_err,SEFLAG_J0510,MAFLAG_J0510,flux_J0520,flux_J0520_err,SEFLAG_J0520,MAFLAG_J0520,flux_J0530,flux_J0530_err,SEFLAG_J0530,MAFLAG_J0530,flux_J0540,flux_J0540_err,SEFLAG_J0540,MAFLAG_J0540,flux_J0550,flux_J0550_err,SEFLAG_J0550,MAFLAG_J0550,flux_J0560,flux_J0560_err,SEFLAG_J0560,MAFLAG_J0560,flux_J0570,flux_J0570_err,SEFLAG_J0570,MAFLAG_J0570,flux_J0580,flux_J0580_err,SEFLAG_J0580,MAFLAG_J0570_1,flux_J0590,flux_J0590_err,SEFLAG_J0590,MAFLAG_J0590,flux_J0600,flux_J0600_err,SEFLAG_J0600,MAFLAG_J0600,flux_J0610,flux_J0610_err,SEFLAG_J0610,MAFLAG_J0610,flux_J0620,flux_J0620_err,SEFLAG_J0620,MAFLAG_J0620,flux_J0630,flux_J0630_err,SEFLAG_J0630,MAFLAG_J0630,flux_J0640,flux_J0640_err,SEFLAG_J0640,MAFLAG_J0640,flux_J0650,flux_J0650_err,SEFLAG_J0650,MAFLAG_J0650,flux_J0660,flux_J0660_err,SEFLAG_J0660,MAFLAG_J0660,flux_J0670,flux_J0670_err,SEFLAG_J0670,MAFLAG_J0670,flux_J0680,flux_J0680_err,SEFLAG_J0680,MAFLAG_J0680,flux_J0690,flux_J0690_err,SEFLAG_J0690,MAFLAG_J0690,flux_J0700,flux_J0700_err,SEFLAG_J0700,MAFLAG_J0700,flux_J0710,flux_J0710_err,SEFLAG_J0710,MAFLAG_J0710,flux_J0720,flux_J0720_err,SEFLAG_J0720,MAFLAG_J0720,flux_J0730,flux_J0730_err,SEFLAG_J0730,MAFLAG_J0730,flux_J0740,flux_J0740_err,SEFLAG_J0740,MAFLAG_J0740,flux_J0750,flux_J0750_err,SEFLAG_J0750,MAFLAG_J0750,flux_J0760,flux_J0760_err,SEFLAG_J0760,MAFLAG_J0760,flux_J0770,flux_J0770_err,SEFLAG_J0770,MAFLAG_J0770,flux_J0780,flux_J0780_err,SEFLAG_J0780,MAFLAG_J0780,flux_J0790,flux_J0790_err,SEFLAG_J0790,MAFLAG_J0790,flux_J0800,flux_J0800_err,SEFLAG_J0800,MAFLAG_J0800,flux_J0810,flux_J0810_err,SEFLAG_J0810,MAFLAG_J0810,flux_J0820,flux_J0820_err,SEFLAG_J0820,MAFLAG_J0820,flux_J0830,flux_J0830_err,SEFLAG_J0830,MAFLAG_J0830,flux_J0840,flux_J0840_err,SEFLAG_J0840,MAFLAG_J0840,flux_J0850,flux_J0850_err,SEFLAG_J0850,MAFLAG_J0850,flux_J0860,flux_J0860_err,SEFLAG_J0860,MAFLAG_J0860,flux_J0870,flux_J0870_err,SEFLAG_J0870,MAFLAG_J0870,flux_J0880,flux_J0880_err,SEFLAG_J0880,MAFLAG_J0880,flux_J0890,flux_J0890_err,SEFLAG_J0890,MAFLAG_J0890,flux_J0900,flux_J0900_err,SEFLAG_J0900,MAFLAG_J0900,flux_J0910,flux_J0910_err,SEFLAG_J0910,MAFLAG_J0910,flux_J1007,flux_J1007_err,SEFLAG_J1007,MAFLAG_J1007,flux_gSDSS,flux_gSDSS_err,SEFLAG_gSDSS,MAFLAG_gSDSS,flux_iSDSS,flux_iSDSS_err,SEFLAG_iSDSS,MAFLAG_iSDSS,flux_rSDSS,flux_rSDSS_err,SEFLAG_rSDSS,MAFLAG_rSDSS,flux_uJAVA,flux_uJAVA_err,SEFLAG_uJAVA,MAFLAG_uJAVA
,,,,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,,mJy,mJy,,
int32,int32,float64,float64,float32,float32,float32,float32,int32,int32,float32,float32,int32,int32,float32,float32,int32,int32,float32,float32,int32,int32,float32,float32,int32,int32,float32,float32,int32,int32,float32,float32,int32,int32,float32,float32,int32,int32,float32,float32,int32,in

In [33]:
miniJPAS_flux[tosave].write('./'+'miniJPAS_flux.fits', format='fits', overwrite=True)